# REFINITIV PEERS AND VALUATION

### Instructions

Please input a valid RIC after running the first and second cell below. Then you can run the rest of the notebook using shift+enter for each cell sequentially. 

This notebook will provide all of the information contained in the popular Eikon/Workspace template of the same name. This includes some general information about the RIC you selected and then various sections on how to calculate Enterprise Value, a financial summary, a financial valuation and finally a drop-down box which allows you to select a 5 year history of over 21 measures.

In [4]:
import refinitiv.data as rd
from refinitiv.data.discovery import Peers
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Box, Layout

%matplotlib inline
rd.open_session()

<refinitiv.data.session.Definition object at 0x13fd8e910 {name='workspace'}>

In [5]:
ric = widgets.Text(
    value='',
    placeholder='Type something',
    description='RIC:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items = [ric]
box = Box(children=items, layout=box_layout)

box

Box(children=(Text(value='', description='RIC:', placeholder='Type something'),), layout=Layout(align_items='s…

### About

In [6]:
df = rd.get_data(ric.value, [
                 "TR.PriceClose.Date", "TR.NetIncome(period=FY0).PED", "TR.NetIncome(period=FI0).PED"])
df.columns = ['Instrument', 'Price Date',
              'Fiscal Year End', 'Fiscal Interim End']
df.transpose()

,0
Instrument,MSFT.O
Price Date,2024-02-16 00:00:00
Fiscal Year End,2023-06-30 00:00:00
Fiscal Interim End,2023-12-31 00:00:00


### General Information

In [7]:
df1 = rd.get_data(ric.value, ['TR.BusinessSummary'])
pd.set_option('display.max_colwidth', None)
df1['Business Description'][0]

'Microsoft Corporation is a technology company. The Company develops and supports software, services, devices, and solutions. The Company’s segments include Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment consists of products and services in its portfolio of productivity, communication, and information services. This segment primarily comprises: Office Commercial, Office Consumer, LinkedIn, and Dynamics business solutions. The Intelligent Cloud segment consists of server products and cloud services, including Azure and other cloud services, SQL Server, Windows Server, Visual Studio, System Center, and related Client Access Licenses (CALs), and Nuance and GitHub; and Enterprise Services, including enterprise support services, industry solutions and Nuance professional services. The More Personal Computing segment primarily comprises Windows, devices, gaming, and search and news advertising.'

### Enterprise Value

In [8]:
df2 = rd.get_data(ric.value, ['TR.NetIncome.PED', 'TR.PriceClose(SDate=0FY)', 'TR.CompanySharesOutstanding', 'TR.CompanyMarketCap', 'TR.CashandSTInvestments',
                              'TR.LTInvestmentAffiliateCompanies', 'TR.TotalDebtOutstanding', 'TR.RedeemablePreferredTtl',
                              'TR.PreferredStockNonRedeemableNet', 'TR.MinorityInterestBSStmt', 'TR.PensionBenefitsUnderfunded'],
                  {'Period': 'FY0'})

df2['TEV'] = df2.iloc[0, 3] + \
    (sum(list(filter(None, df2.iloc[0, 6:10]))) -
     sum(list(filter(None, df2.iloc[0, 4:5]))))

df3 = rd.get_data(ric.value, ['TR.NetIncome.PED', 'TR.PriceClose', 'TR.CompanySharesOutstanding', 'TR.CompanyMarketCap', 'TR.CashandSTInvestments',
                              'TR.LTInvestmentAffiliateCompanies', 'TR.TotalDebtOutstanding', 'TR.RedeemablePreferredTtl',
                              'TR.PreferredStockNonRedeemableNet', 'TR.MinorityInterestBSStmt', 'TR.PensionBenefitsUnderfunded'],
                  {'Period': 'LTM'})

df3['TEV'] = df3.iloc[0, 3] + \
    (sum(list(filter(None, df3.iloc[0, 6:10]))) -
     sum(list(filter(None, df3.iloc[0, 4:5]))))

df2 = pd.concat([df2, df3])
df2.set_index('Period End Date', inplace=True)
df2.transpose()

Period End Date,2023-06-30,2024-02-20
Instrument,MSFT.O,MSFT.O
Price Close,376.04,404.06
Company Shares,7430436229,7430436229
Company Market Cap,3002342062689.740234,3002342062689.740234
Cash and Short Term Investments,111262000000,110164250000
LT Investment - Affiliate Companies,9879000000,11021000000
Total Debt,64304000000,78262250000
"Redeemable Preferred Stock, Total",,
"Preferred Stock - Non Redeemable, Net",,
Minority Interest,,


### Financial Summary

In [9]:
df4 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FI0).rfperiod', 'TR.TotalRevenue(period=FI0)', 'PERCENT_CHG(TR.TotalRevenue(period=FI0),TR.TotalRevenue(period=FI-1))',
                              'TR.GrossProfit(period=FI0)', 'TR.GrossProfit(period=FI0)/TR.TotalRevenue(period=FI0)',
                              'TR.EBITDA(period=FI0)', 'TR.EBITDAMarginPercent(period=FI0)',
                              'TR.EBIT(period=FI0)', 'TR.EBITMarginPercent(period=FI0)',
                              'TR.NetIncome(period=FI0)', 'TR.NetProfitMargin(period=FI0)',
                              'TR.CapitalExpendituresCFStmt(period=FI0)', 'TR.CapitalExpendituresCFStmt(period=FI0)/TR.TotalRevenue(period=FI0)'])

df4.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df5 = rd.get_data(ric.value, ['TR.TotalRevenue(period=LTM).rfperiod', 'TR.TotalRevenue(period=LTM)', 'PERCENT_CHG(TR.TotalRevenue(period=FI0),TR.TotalRevenue(period=FI-4))',
                              'TR.GrossProfit(period=LTM)', 'TR.GrossMargin(period=LTM)',
                              'TR.EBITDATTM', 'TR.EBITDAMarginPercent(period=LTM)',
                              'TR.EBITTTM', 'TR.EBITMarginPercent(period=LTM)',
                              'TR.NetIncome(period=LTM)', 'TR.NetProfitMargin(period=LTM)',
                              'TR.CapitalExpendituresCFStmt(period=LTM)', 'TR.CapitalExpendituresCFStmt(period=LTM)/TR.TotalRevenue(period=LTM)'])

df5.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df6 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FY-1).rfperiod', 'TR.TotalRevenue(period=FY-1)', 'PERCENT_CHG(TR.TotalRevenue(period=FY-1),TR.TotalRevenue(period=FY-2))',
                              'TR.GrossProfit(period=FY-1)', 'TR.GrossMargin(period=FY-1)',
                              'TR.EBITDA(period=FY-1)', 'TR.EBITDAMarginPercent(period=FY-1)',
                              'TR.EBIT(period=FY-1)', 'TR.EBITMarginPercent(period=FY-1)',
                              'TR.NetIncome(period=FY-1)', 'TR.NetProfitMargin(period=FY-1)',
                              'TR.CapitalExpendituresCFStmt(period=FY-1)', 'TR.CapitalExpendituresCFStmt(period=FY-1)/TR.TotalRevenue(period=FY-1)'])

df6.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']


df7 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FY0).rfperiod', 'TR.TotalRevenue(period=FY0)', 'PERCENT_CHG(TR.TotalRevenue(period=FY0),TR.TotalRevenue(period=FY-1))',
                              'TR.GrossProfit(period=FY0)', 'TR.GrossMargin(period=FY0)',
                              'TR.EBITDA(period=FY0)', 'TR.EBITDAMarginPercent(period=FY0)',
                              'TR.EBIT(period=FY0)', 'TR.EBITMarginPercent(period=FY0)',
                              'TR.NetIncome(period=FY0)', 'TR.NetProfitMargin(period=FY0)',
                              'TR.CapitalExpendituresCFStmt(period=FY0)', 'TR.CapitalExpendituresCFStmt(period=FY0)/TR.TotalRevenue(period=FY0)'])

df7.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df8 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY1).rfperiod', 'TR.RevenueMeanEstimate(period=FY1)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY1),TR.TotalRevenue(period=FY0))",
                              'TR.GrossIncomeMean(period=FY1)', 'TR.GPMMean(period=FY1)',
                              'TR.EBITDAMean(period=FY1)', "TR.EBITDAMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.EBITMean(period=FY1)', "TR.EBITMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.NetIncomeMeanEstimate(period=FY1)', "TR.NetIncomeMeanEstimate(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.CAPEXMean(period=FY1)', "TR.CAPEXMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100"])

df8.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df9 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY2).rfperiod', 'TR.RevenueMeanEstimate(period=FY2)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY2),TR.TotalRevenue(period=FY1))",
                              'TR.GrossIncomeMean(period=FY2)', 'TR.GPMMean(period=FY2)',
                              'TR.EBITDAMean(period=FY2)', "TR.EBITDAMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.EBITMean(period=FY2)', "TR.EBITMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.NetIncomeMeanEstimate(period=FY2)', "TR.NetIncomeMeanEstimate(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.CAPEXMean(period=FY2)', "TR.CAPEXMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100"])

df9.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df10 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY3).rfperiod', 'TR.RevenueMeanEstimate(period=FY3)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY3),TR.TotalRevenue(period=FY2))",
                               'TR.GrossIncomeMean(period=FY3)', 'TR.GPMMean(period=FY3)',
                               'TR.EBITDAMean(period=FY3)', "TR.EBITDAMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.EBITMean(period=FY3)', "TR.EBITMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.NetIncomeMeanEstimate(period=FY3)', "TR.NetIncomeMeanEstimate(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.CAPEXMean(period=FY3)', "TR.CAPEXMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100"])

df10.columns = ['Instrument', 'Fiscal Period', 'Revenue',
                '  Growth(%)', 'Gross Income', '  Margin(%)', 'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df4 = pd.concat([df4, df5, df6, df7, df8, df9, df10], ignore_index=True)
df4.set_index('Fiscal Period', inplace=True)
df4.transpose()

Fiscal Period,FQ0,LTM,FY-1,FY0,FY1,FY2,FY3
Instrument,MSFT.O,MSFT.O,MSFT.O,MSFT.O,MSFT.O,MSFT.O,MSFT.O
Revenue,62020000000,227583000000,198270000000,211915000000,244101194980,278743387680,318541706150
Growth(%),9.736893,17.580147,17.956071,6.88203,15.188257,NaN,NaN
Gross Income,42397000000,158735000000,135620000000,146052000000,169192622500,191256615710,218894048000
Margin(%),0.683602,69.778272,68.40167,68.92009,69.29733,68.7331,68.96222
EBITDA,32991000000,117836000000,97983000000,103555000000,128684629270,148481081400,171453674000
Margin(%),53.194131,51.695241,49.418974,48.866291,52.717738,53.268019,53.824561
EBIT,27032000000.0,100533000000.0,83383000000.0,89693999999.999893,107017998600.0,122284811180.0,141811414810.0
Margin(%),43.58594,44.156526,42.055278,42.325461,43.841653,43.870031,44.518947
Net Income,21870000000,82541000000,72738000000,72361000000,86829650500,99030025100,115523868350


### Financial Valuation

In [10]:
headers = ['FY0', 'LTM', 'FY1', 'FY2', 'FY3']
index = ['a', 'b']
TEV_to_Sales = pd.Series([df2['TEV'][0]/df4.iloc[3, 1], df2['TEV'][1]/df4.iloc[1, 1], df2['TEV'][1]/df4.iloc[4, 1],
                          df2['TEV'][1]/df4.iloc[5, 1], df2['TEV'][1]/df4.iloc[6, 1]], index=headers)
TEV_to_EBITDA = pd.Series([df2['TEV'][0]/df4.iloc[3, 5], df2['TEV'][1]/df4.iloc[1, 5], df2['TEV'][1]/df4.iloc[4, 5],
                           df2['TEV'][1]/df4.iloc[5, 5], df2['TEV'][1]/df4.iloc[6, 5]], index=headers)
TEV_to_EBIT = pd.Series([df2['TEV'][0]/df4.iloc[3, 7], df2['TEV'][1]/df4.iloc[1, 7], df2['TEV'][1]/df4.iloc[4, 7],
                         df2['TEV'][1]/df4.iloc[5, 7], df2['TEV'][1]/df4.iloc[6, 7]], index=headers)
list_of_series = [TEV_to_Sales, TEV_to_EBITDA, TEV_to_EBIT]
tev = pd.DataFrame(list_of_series, columns=headers)
tev.rename(index={0: 'TEV_to_Sales', 1: 'TEV_to_EBITDA', 2: 'TEV_to_EBIT'})

,FY0,LTM,FY1,FY2,FY3
TEV_to_Sales,-13.782548,-12.767335,-11.903376,-10.424026,-9.121658
TEV_to_EBITDA,-28.204612,-24.658240,-22.579452,-19.569014,-16.947017
TEV_to_EBIT,-32.563255,-28.902235,-27.150838,-23.761155,-20.489383


In [11]:
curn = rd.get_data(ric.value, ['TR.PriceClose.currency'])
curn = curn['Currency'][0].upper()
curn

df11 = rd.get_data(ric.value, ['TR.EBITDA(period=FY0).rfperiod', "TR.EBITDA(period=FY0)/TR.InterestExpenseSupplemental(period=FY0)",
                                    "TR.EBIT(period=FY0)/TR.InterestExpenseSupplemental(period=FY0)",
                                    "(TR.EBITDA(period=FY0)-TR.CapitalExpendituresCFStmt(period=FY0))/(TR.InterestExpenseSupplemental(period=FY0))",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EBITDA(period=FY0)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EV*100",
                                    "TR.PriceClose('"+df2.index[0].strftime('%Y-%m-%d')+"',curn="+curn+")/TR.DilutedEpsExclExtra(Period=FY0)"])

df11.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df12 = rd.get_data(ric.value, ['TR.EBITDA(period=LTM).rfperiod', "TR.EBITDA(period=LTM)/TR.InterestExpenseSupplemental(period=LTM)",
                                    "TR.EBIT(period=LTM)/TR.InterestExpenseSupplemental(period=LTM)",
                                    "(TR.EBITDA(period=LTM)-TR.CapitalExpendituresCFStmt(period=LTM))/(TR.InterestExpenseSupplemental(period=LTM))",
                                    "TR.TotalDebtOutstanding(period=LTM)/TR.EBITDA(period=LTM)",
                                    "TR.TotalDebtOutstanding(period=LTM)/TR.EV*100",
                                    "TR.PriceClose('"+df2.index[0].strftime('%Y-%m-%d')+"',curn="+curn+")/TR.DilutedEpsExclExtra(Period=LTM)"])

df12.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']


df13 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY1).rfperiod', "TR.EBITDAMean(period=FY1)/TR.IntExpMean(period=FY1)",
                                    "TR.EBITMean(period=FY1)/TR.IntExpMean(period=FY1)",
                                    "(TR.EBITMean(period=FY1)-TR.CAPEXMean(period=FY1))/TR.IntExpMean(period=FY1)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EBITDAMean(period=FY1)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY1)"])

df13.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df14 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY2).rfperiod', "TR.EBITDAMean(period=FY2)/TR.IntExpMean(period=FY2)",
                                    "TR.EBITMean(period=FY2)/TR.IntExpMean(period=FY2)",
                                    "(TR.EBITMean(period=FY2)-TR.CAPEXMean(period=FY2))/TR.IntExpMean(period=FY2)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EBITDAMean(period=FY2)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY2)"])

df14.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df15 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY3).rfperiod', "TR.EBITDAMean(period=FY3)/TR.IntExpMean(period=FY3)",
                                    "TR.EBITMean(period=FY3)/TR.IntExpMean(period=FY3)",
                                    "(TR.EBITMean(period=FY3)-TR.CAPEXMean(period=FY3))/TR.IntExpMean(period=FY3)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EBITDAMean(period=FY3)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY0)"])

df15.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df11 = pd.concat([df11, df12, df13, df14, df15], ignore_index=True)
df11.set_index('Fiscal Period', inplace=True)

df11.transpose()

Fiscal Period,FY0,LTM,FY1,FY2,FY3
Instrument,MSFT.O,MSFT.O,MSFT.O,MSFT.O,MSFT.O
EBITDA/Interest Expense,52.619411,48.854063,131.502541,126.999947,370.510371
EBIT/Interest Expense,45.57622,41.680348,109.361458,104.593558,306.453625
EBITDA-Capex/Interest Expense,38.337398,34.259536,66.603639,65.450276,198.816021
Total Debt/EBITDA,0.620965,0.664162,0.499702,0.433079,0.375052
Total Debt/EV,2.132279,2.595125,2.132279,2.132279,2.132279
PE,35.164168,30.79845,34.67463,30.275579,41.74277


In [12]:
curn = rd.get_data(ric.value, ['TR.PriceClose.currency'])
curn = curn['Currency'][0].upper()
curn

'USD'

In [13]:
mSel = widgets.Dropdown(
    options=[('Book Value per Share', 'BVPS'), ('Capital Expenditure', 'Capex'), ('Cash Earnings per Share', 'EPSCash'),
             ('Cash Flow per Share', 'CFPS'), ('Dividend per Share',
                                               'DPS'), ('EBIT', 'EBIT'), ('EBITDA', 'EBITDA'),
             ('EBITDA per Share', 'EBITDAPS'), ('Enterprise Value',
                                                'EV'), ('EPS', 'EPS'), ('EPS -Fully Reported', 'EPSFR'),
             ('Funds Flow from Operations', 'FFO'), ('Gross Margin',
                                                     'GPM'), ('Net Asset Value', 'NetAssetValue'),
             ('Net Debt', 'NetDebt'), ('Net Income',
                                       'NetDebt'), ('Operating Profit', 'OProfit'),
             ('Pre-tax Profit', 'PreTaxProfit'), ('Revenue', 'Revenue'), ('ROA', 'ROA'), ('ROE', 'ROE')],
    value='BVPS',
    description='Metric:',
    disabled=False
)

pSel = widgets.Dropdown(
    options=['Annual', 'Interim'],
    value='Annual',
    description='Period:',
    disabled=False
)

cSel = widgets.Dropdown(
    options=['USD', 'EUR', 'GBP', 'CAD', 'RUB', 'CNY', 'INR', 'JPY'],
    value='GBP',
    description='Curn:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items2 = [mSel, pSel, cSel]
box2 = Box(children=items2, layout=box_layout)
box2

Box(children=(Dropdown(description='Metric:', options=(('Book Value per Share', 'BVPS'), ('Capital Expenditure…

In [14]:
if pSel.value == 'Annual':
    df16 = rd.get_data(ric.value, [ 'TR.'+mSel.value+'ActReportDate',
                                        'TR.'+mSel.value+'MeanEstimate', 'TR.'+mSel.value+'ActValue',
                                        'TR.'+mSel.value+'ActSurprise', 'TR.'+mSel.value+'ActSueScore',
                                        'TR.'+mSel.value+'NumofEstimates'], {'Period': 'FY0', 'Sdate': 'FY0', 'Edate': 'FY-4',
                                                                             'FRQ': 'FY', 'Curn': cSel.value})
else:
    df16 = rd.get_data(ric.value, ['TR.'+mSel.value+'ActReportDate',
                                        'TR.'+mSel.value+'MeanEstimate', 'TR.'+mSel.value+'ActValue',
                                        'TR.'+mSel.value+'ActSurprise', 'TR.'+mSel.value+'ActSueScore',
                                        'TR.'+mSel.value+'NumofEstimates'], {'Period': 'FI0', 'Sdate': 'FI0', 'Edate': 'FI-19',
                                                                             'FRQ': 'FI', 'Curn': cSel.value})
df16

,Instrument,Report Date,Book Value Per Share - Actual,Book Value Per Share - Actual Surprise,Book Value Per Share - Standard Unexpected Earnings,Book Value Per Share - Number of Estimates
0,MSFT.O,2023-07-25 16:01:00,21.509957,-5.463,-0.31088,9
1,MSFT.O,2022-07-26 16:09:00,18.555241,-9.694,-0.92111,11
2,MSFT.O,2021-07-27 16:02:00,13.608138,-1.204,-0.3386,3
3,MSFT.O,2020-07-22 16:03:00,12.276115,0.34,0.09952,5
4,MSFT.O,2019-07-18 16:07:00,10.671747,7.94,1.39941,7


In [15]:
rd.close_session()

**Further Resources for Refinitiv Data Libraries API on Developer Community Portal**

* [Overview](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) 
* [Quick Start ](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start)
* [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
* [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials)
* [Q&A Forums](https://community.developers.refinitiv.com/spaces/321/index.html)